In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.3 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 8.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 64.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 30.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 6.5 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-km26knuc/overlay/local/bin
    changing mode of /tmp/pip-build-env-km26knuc/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-km26knuc/overlay/lo

In [2]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [4]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [5]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [6]:
from google.colab import files
pd= files.upload()

Saving processed_reviews_128 (6).csv to processed_reviews_128 (6).csv


In [7]:
import pandas as pd
import random
df = pd.read_csv('processed_reviews_128 (6).csv')


In [8]:
df = df[['sentiment', 'cleaned_review']]
df.head()

,sentiment,cleaned_review
0,positive,mani funni scene peopl dont normal pay attent ...
1,negative,shame shame fine actor joseph fienn would allo...
2,negative,movi pretti predict nuff saidfrom delay kiss s...
3,positive,black mask jet li play bioengin superkil turn ...
4,negative,watch entir movi recogn particip william hurt ...


In [12]:
df_incorrect = df.loc[df['sentiment'] == 'negative']['cleaned_review'].reset_index(drop=True)
df_correct = df.loc[df['sentiment'] == 'positive']['cleaned_review'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'best show sinc seinfeld she realli realli funni total self centered hulk gay stoner neighbor departur song cartoon make freshest show tv one show make point watch scene wise old black ladi drugstor oh wait your close look old turn face finger walk away lol cough syrup overdos sleep god funni stupid lot fun show weak point sister coponli theyr darn normal realli cant wait next show someth havent felt show long time'

In [13]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'clich ridden worst romant comedi ever seen everi scene cring worthi two lead actor corey danni soo annoy corey dumb naiv never listen danni fals promisesbr br neve campbel killer urban legend redeem qualiti poor attempt film danni dean para look late thirti girl he tri bed corey look she still collegebr br australia film call smooth noth smooth film avoid'

In [14]:
df_correct_ = 'peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen'
df_incorrect_ = 'read review decid rent dvd version br br like classic music want learn bach br br disappoint guess know enough bach music comment enough understand import music play br br mayb would appropri guidanc expert bach music explain film br br realli tri saw whole film hop would abl enjoy least br br see risk'

In [ ]:
#Step 6: Generating the Response

In [18]:
def generate_prompt(df, index):
    return f'''Example of movie review ratings:
    Review: {df_correct_}
     ANSWER: positive

    Please read and analyze both reviews comprehensively. Evaluate the similarity between these two reviews based on their content. Conclude with a statement on whether the reviews are similar or not, and explain the reasoning behind your conclusion. :
    Review: "{df['cleaned_review'][index][:128]}"
    '''
START_INDEX = 1
END_INDEX = 50
review_index = START_INDEX

prompts = []
model_r = []
model_responses = []
review_index = 0

while review_index < END_INDEX:
    prompt = generate_prompt(df, review_index)

    prompt_template = f'''SYSTEM:You are a movie review evaluator. Your task is to evaluate the similarity between these two reviews.

    USER: {prompt}

    ASSISTANT:
    '''

    response = lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)

    print(response)
    print(response["choices"][0]["text"])

    response_text = response["choices"][0]["text"]

    matches_ = re.search(r'ASSISTANT:(.*?)(?=Model response|$)', response_text, re.DOTALL)

    if matches_:
        assistant_response = matches_.group(1).strip()
        prompts.append(prompt_template)
        model_r.append(assistant_response)

        answer_match = re.search(r'\b0(?:\.\d+)?|1(?:\.0+)?\b', assistant_response)
        model_responses.append(answer_match.group(0) if answer_match else None)
    else:
        print("Model response does not contain a valid assistant response. Retrying...")

    review_index += 1


df_results = pd.DataFrame({
    'Prompt': prompts,
    'Model': model_r,
    'ModelResponse': model_responses,
})




{'id': 'cmpl-c7e23b57-20c2-498f-8878-f7bd7b4a8877', 'object': 'text_completion', 'created': 1703156621, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to evaluate the similarity between these two reviews.\n\n    USER: Example of movie review ratings:\n    Review: peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen\n     ANSWER: positive\n    \n    Please read and analyze both reviews comprehensively. Evaluate the similarity between these two reviews based on their content. Conclude with a statement on whether the

Llama.generate: prefix-match hit


{'id': 'cmpl-45ba5cbc-3d2b-478d-a7ec-e097d0316a72', 'object': 'text_completion', 'created': 1703156680, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to evaluate the similarity between these two reviews.\n\n    USER: Example of movie review ratings:\n    Review: peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen\n     ANSWER: positive\n    \n    Please read and analyze both reviews comprehensively. Evaluate the similarity between these two reviews based on their content. Conclude with a statement on whether the

Llama.generate: prefix-match hit


{'id': 'cmpl-8cb2f498-644d-4c68-b04a-92ac395853d1', 'object': 'text_completion', 'created': 1703156740, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to evaluate the similarity between these two reviews.\n\n    USER: Example of movie review ratings:\n    Review: peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen\n     ANSWER: positive\n    \n    Please read and analyze both reviews comprehensively. Evaluate the similarity between these two reviews based on their content. Conclude with a statement on whether the

...

{'id': 'cmpl-1f0eb53b-7549-4762-8960-5bccdfd79fee', 'object': 'text_completion', 'created': 1703159094, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to evaluate the similarity between these two reviews.\n\n    USER: Example of movie review ratings:\n    Review: peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen\n     ANSWER: positive\n    \n    Please read and analyze both reviews comprehensively. Evaluate the similarity between these two reviews based on their content. Conclude with a statement on whether the

Llama.generate: prefix-match hit


{'id': 'cmpl-4f522b0c-0d4d-4324-b53f-aa27e1a2830c', 'object': 'text_completion', 'created': 1703159153, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to evaluate the similarity between these two reviews.\n\n    USER: Example of movie review ratings:\n    Review: peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen\n     ANSWER: positive\n    \n    Please read and analyze both reviews comprehensively. Evaluate the similarity between these two reviews based on their content. Conclude with a statement on whether the

Llama.generate: prefix-match hit


{'id': 'cmpl-5d848e53-dc7c-47b6-8671-7fc1388badce', 'object': 'text_completion', 'created': 1703159212, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': 'SYSTEM:You are a movie review evaluator. Your task is to evaluate the similarity between these two reviews.\n\n    USER: Example of movie review ratings:\n    Review: peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen\n     ANSWER: positive\n    \n    Please read and analyze both reviews comprehensively. Evaluate the similarity between these two reviews based on their content. Conclude with a statement on whether the

In [19]:
df_results

,Prompt,Model,ModelResponse
0,SYSTEM:You are a movie review evaluator. Your ...,"Sure, I'd be happy to help! Here are the two r...",1
1,SYSTEM:You are a movie review evaluator. Your ...,USER: I apologize for the mistake in my previo...,None
2,SYSTEM:You are a movie review evaluator. Your ...,"Based on my analysis, both reviews are similar...",1
3,SYSTEM:You are a movie review evaluator. Your ...,USER: Hey there! I'm looking for a movie that ...,None
4,SYSTEM:You are a movie review evaluator. Your ...,"Based on my analysis, the two reviews are simi...",None
5,SYSTEM:You are a movie review evaluator. Your ...,Sure! I'd be happy to help you evaluate the si...,1
6,SYSTEM:You are a movie review evaluator. Your ...,Sure! I'd be happy to help you evaluate the si...,None
7,SYSTEM:You are a movie review evaluator. Your ...,"Based on my analysis, the two reviews are not ...",1
8,SYSTEM:You are a movie review evaluator. Your ...,Sure! I'd be happy to help you evaluate the si...,None
9,SYSTEM:You are a movie review evaluator. Your ...,Sure! I'd be happy to help you evaluate the si...,1


In [20]:
df_results.to_csv('responses50_100.csv', index=False)
files.download('responses50_100.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install scikit-learn

true_labels = [0,1,0,1,0,1,1,1,1,0]
predicted_labels = [1,1,1,0,1,1,1,1,1,1]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 50.00%
